参考文献《Julia 高性能科学计算 第2版》
## 1 Julia 语言基础
### 1.1 向量、矩阵和数组
Julia中矩阵的定义与Matlab类似，分号表示换行。例如：

In [1]:
a = [1; 2; 3]

3-element Vector{Int64}:
 1
 2
 3

In [2]:
b = [4 5 6]

1×3 Matrix{Int64}:
 4  5  6

In [3]:
A = [1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

可以用索引[i,j]来访问A中的元素(i,j)。并且与Matlab类似，矩阵的**索引从1开始**，不是从0开始。

In [4]:
A[1, 3]

3

常见的矩阵操作，如转置：

In [5]:
transpose(A)

3×2 transpose(::Matrix{Int64}) with eltype Int64:
 1  4
 2  5
 3  6

In [6]:
A'

3×2 adjoint(::Matrix{Int64}) with eltype Int64:
 1  4
 2  5
 3  6

对于矢量点积，可以用Julia自带的方式

In [7]:
a = [1; 2; 3]
b = [7; 8; 9]
a'*b

50

也可以采用官方的线性代数库

In [8]:
using LinearAlgebra
dot(a, b)

50

指定大小的单位矩阵,(中文译作特定大小的恒等矩阵，应该是笔误，identity matrix 指单位阵)

In [9]:
Matrix(1.0I,3,3)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

这里，对角数值可以是任意数，且维度不必是方阵，比如

In [10]:
Matrix(2.0I,3,4)

3×4 Matrix{Float64}:
 2.0  0.0  0.0  0.0
 0.0  2.0  0.0  0.0
 0.0  0.0  2.0  0.0

In [11]:
Matrix(0.1I,4,2)

4×2 Matrix{Float64}:
 0.1  0.0
 0.0  0.1
 0.0  0.0
 0.0  0.0

元素均为0或1的矩阵，定义与Matlab类似，比如

In [12]:
ones(3,2)

3×2 Matrix{Float64}:
 1.0  1.0
 1.0  1.0
 1.0  1.0

In [13]:
zeros(4,5)

4×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

矩阵的逆矩阵

In [14]:
A = [1 3 2; 3 2 2; 1 1 1]
inv(A)

3×3 Matrix{Float64}:
  1.11022e-16   1.0  -2.0
  1.0           1.0  -4.0
 -1.0          -2.0   7.0

数值方法存在一些微小的误差

In [15]:
inv(A)*A

3×3 Matrix{Float64}:
  1.0          0.0  -4.44089e-16
 -8.88178e-16  1.0  -8.88178e-16
  0.0          0.0   1.0

可以发现由于求逆存在误差，因此两者乘积不是严格的单位阵。通常情况下，矩阵元素的数据类型是根据赋值自动指定的，但是有些时候往往需要先生成特定大小的矩阵，后期再具体赋值。可以在Array中通过undef关键词来生成空的矩阵，比如

In [16]:
d = Array{Float64}(undef, 3)

3-element Vector{Float64}:
 1.225529676e-315
 1.22855067e-315
 5.434431220789e-311

可以看到一些接近零的数值被预分配给了矩阵的元素，之后可以根据需要给数组元素赋新的数值。

In [17]:
d[1] = 1
d

3-element Vector{Float64}:
 1.0
 1.22855067e-315
 5.434431220789e-311

可以看到，尽管赋值时给了整数1，但程序给定的数值类型仍然是之前定义的Float64。对于需要进行矩阵运算的列向量或行向量，应该显式地指定另一个维度值1，比如

In [18]:
c = Array{Float64}(undef, 3, 1)
d = Array{Float64}(undef, 1, 3)
c * d

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [19]:
d*c

1×1 Matrix{Float64}:
 0.0

### 1.2 元组

In [20]:
数组的元素可以是一对数据类型，例如想在数组中存储(1,2)、(2,3)、(3,4)，我们可以直接创建如下(Int64,Int64)类型的数组。

LoadError: syntax: invalid character "，" near column 29

In [ ]:
pairs = Array{Tuple{Int64,Int64}}(undef,3)

3-element Array{Tuple{Int64,Int64},1}:
 (377403056, 163125360)
 (376811984, 398360608)
 (377403072, 398328192)

In [ ]:
pairs[1] = (1,2)

(1, 2)

In [ ]:
pairs

3-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (376811984, 398360608)
 (377403072, 398328192)

也可以通过直接赋值的方式来定义，如

In [ ]:
newPairs = [(1,2); (2,3); (3,4)]

3-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (2, 3)
 (3, 4)

这种数据类型在处理具有节点和链接的网络数据时非常有用，比如存储像(i,j)这样的数据。(类似与关节的内外接物体$\alpha$和$\beta$)
### 1.3 索引和范围

对于如下的矢量

In [ ]:
a = [10; 20; 30; 40; 50; 60; 70; 80; 90]

9-element Array{Int64,1}:
 10
 20
 30
 40
 50
 60
 70
 80
 90

可以指定某几个连续元素

In [ ]:
a[1:3]

3-element Array{Int64,1}:
 10
 20
 30

也可以等间隔取元素，从第1个开始，每隔3个取1个，到9为止，因此取了a中的元素1，4，7.

In [ ]:
a[1:3:9]

3-element Array{Int64,1}:
 10
 40
 70

此外还有end关键字，例如

In [ ]:
a[end-2:end]

3-element Array{Int64,1}:
 70
 80
 90

同时支持部分赋值

In [ ]:
b = [1;4;7]
a[1:3:9] = b
c = [2;5;8]
a[end-2:end] = c
a

9-element Array{Int64,1}:
  1
 20
 30
  4
 50
 60
  2
  5
  8

可以用范围来定义数组，只要把范围放入collect函数即可，如下

In [ ]:
c = collect(1:2:9)

5-element Array{Int64,1}:
 1
 3
 5
 7
 9

对于矩阵$\underline{A}$

In [ ]:
A = [1 2 3; 4 5 6; 7 8 9]

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

访问列：

In [ ]:
A[:,2]

3-element Array{Int64,1}:
 2
 5
 8

In [ ]:
A[:,2:3]

3×2 Array{Int64,2}:
 2  3
 5  6
 8  9

访问行：

In [ ]:
A[3,:]

3-element Array{Int64,1}:
 7
 8
 9

**注意**：此时发现返回值为一列向量，为了获得行向量，可以这样写：

In [ ]:
A[3:3, :]

1×3 Array{Int64,2}:
 7  8  9

In [ ]:
A[1:2, :]

2×3 Array{Int64,2}:
 1  2  3
 4  5  6

### 1.4 打印信息
最常见的打印函数是println()和print()

In [ ]:
a = 123.0
b = [1; 3; 10]
print(a)
print(b)
println(a)
println(b)

123.0[1, 3, 10]123.0
[1, 3, 10]


对于数组，有：

In [ ]:
println("b is $b.")

b is [1, 3, 10].


In [ ]:
println("The first element of b is $(b[1]).")

The first element of b is 1.


更高级的功能可以直接调用C的printf形式，如下：

In [ ]:
using Printf
@printf("The %s of a = %f","value",a)

The value of a = 123.000000

当需要打印位数不同的一系列数字，并希望它们以某种方式对齐时，@printf宏非常有用，比如：

In [ ]:
c = [123.12345;
10.983;
1.0932132]
for i in 1:length(c)
    println("c[$i] = $(c[i])")
end
for i in 1:length(c)
    @printf("c[%d] = %7.3f\n", i, c[i])
end

c[1] = 123.12345
c[2] = 10.983
c[3] = 1.0932132
c[1] = 123.123
c[2] =  10.983
c[3] =   1.093


可以发现@printf输出更加整齐美观，%7.3表示希望总位数为7(包含小数点)，小数点后位数为3，\n表示换行。此外，还有一个名为@sprintf的宏，功能基本相同，但是它返回一个字符串，而不是打印在屏幕上。如下所示：

In [ ]:
str = @sprintf("The %s of a = %f", "value", a)
println(str)

The value of a = 123.000000


### 1.5 集合、字典和循环
最常见的是for循环
```[julia]
for i in i
# 对每个i执行一定操作
end
```
此外，若在某些点中止，可以用break打破循环。例如：

In [ ]:
for i in 1:5  
    if i>3
        break end
    println(i)
    end

1
2
3


此处的范围1:5是一个集合，另一个非常有用的集合是字典。每个字典包含一个key和一个value，一一对应。如下：

In [ ]:
my_keys = ["Zinedine Zidane", "Magic Johnson", "Yuna Kim"]
my_values = ["football", "basketball", "figure skating"]
d = Dict()
for i in 1:length(my_keys)
    d[my_keys[i]] = my_values[i]
end
d

Dict{Any,Any} with 3 entries:
  "Magic Johnson"   => "basketball"
  "Zinedine Zidane" => "football"
  "Yuna Kim"        => "figure skating"

由上得出两点：
- key 与 value 是一一对应的关系
- 具体各对数据在字典中的顺序不重要，如果不同组之间有顺序关系时，慎用。可以直接添加新元素，如：

In [ ]:
d["Diego Maradona"] = "football"
d

Dict{Any,Any} with 4 entries:
  "Magic Johnson"   => "basketball"
  "Zinedine Zidane" => "football"
  "Diego Maradona"  => "football"
  "Yuna Kim"        => "figure skating"

加了一个while循环的例子，原书没有，简单参考

In [ ]:
i = 0
while i<10
    println(i)
    i += 1
end

0
1
2
3
4
5
6
7
8
9


### 1.6 函数
创建函数的基本方式如下：

In [ ]:
function f(x,y)
    return 3x + y
end

f (generic function with 1 method)

调用函数：

In [ ]:
f(3,1)

10

In [ ]:
3 * (f(3,2) + f(9,6))

132

对于简单的函数，甚至可以直接赋值。比如：

In [ ]:
g(x, y) = 3x + y

g (generic function with 1 method)

In [ ]:
g(3,1)

10

函数可以有多个返回值，例如：

In [ ]:
function my_func(n, m)
    a = zeros(n,1)
    b = ones(m,1)
    return a, b
end

my_func (generic function with 1 method)

In [ ]:
x, y = my_func(3,5)

([0.0; 0.0; 0.0], [1.0; 1.0; … ; 1.0; 1.0])

In [ ]:
x

3×1 Matrix{Float64}:
 0.0
 0.0
 0.0

In [ ]:
y

5×1 Matrix{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0

对于自变量是标量的某一函数f,可以用f.来进行矢量运算，例如对于标量的开方函数sqrt(),有：

In [ ]:
sqrt(9)

3.0

In [ ]:
sqrt.([9 16])

1×2 Matrix{Float64}:
 3.0  4.0

### 1.7 变量的作用域
如下代码

In [ ]:
function f(x)
    return x+2
end
function g(x)
    return x+3
end

g (generic function with 2 methods)

在这两个函数中，都用到了变量x,但是这两个x并不冲突，因为他们被定义在了不同的作用域块中，典型的作用域块有函数、for循环和while循环。
当一个变量被定义或首次引入时，该变量在所在作用域块中可用。

为更好控制变量的作用域，可以使用global、local和const这样的关键词。

### 1.8 随机数生成
简单调用rand()可以生成一个从0到1之间均匀分布的随机数。

In [21]:
rand()

0.6679200625652943

也可以生成含有随机元素的向量，比如生成一个含有3个0到1之间的随机元素的向量。

In [22]:
rand(3)

3-element Vector{Float64}:
 0.4667985711358458
 0.13482300648118417
 0.5482871189680121

或者创建随机矩阵。

In [23]:
rand(3,2)

3×2 Matrix{Float64}:
 0.960633  0.850998
 0.310951  0.353916
 0.613157  0.28566

[0,100]区间内均匀分布的随机数向量。

In [31]:
rand(4)*100

4-element Vector{Float64}:
 27.395587163993007
 77.62518191005995
 90.70803046419987
 18.923307701335844

In [1]:
include("testjl.jl")
addtest(1,3)

7